<a href="https://colab.research.google.com/github/Ilmaj17/Hotel_Risk_Classifier/blob/main/Hotel_Risk_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [440]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


###Determining whether it is a technical issue or a issue related to service quality

In [441]:
df = pd.read_csv("/content/drive/MyDrive/Data Sets/Capsule Hotel Risk.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          700 non-null    int64 
 1   content             700 non-null    object
 2   translated_content  700 non-null    object
 3   priority_score      700 non-null    int64 
 4   problem_domain      700 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 27.5+ KB


In [442]:
df.head(5)

,Unnamed: 0,content,translated_content,priority_score,problem_domain
0,1,"Sorry, including the expensive price + apk usi...","Sorry, including the expensive price + apk usi...",2,0
1,2,Setelah install terus pas buka aplikasi ngebla...,"After installed, the app stuck with white display",3,1
2,3,"gak bisa submit pada saat verifikasi data, pad...","Cannot submit the data verification, even my i...",3,1
3,4,Aplilasi restart mulu,The app keep restarting,3,1
4,5,"udah mahal , makanan ga enak, banyak lalat","Overpriced, bad food, too many flies",2,0


###Pipeline


*   Remove any unwanted columns
*   Converting categorical columns to numerical columns
*   Classification Model





In [443]:
from sklearn.base import BaseEstimator, TransformerMixin

In [444]:
#Creating an estimator by extending BaseEstimator class
class NameDropper(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X.drop(['content'], axis=1)

df2 = df
#dropper = NameDropper()
#df2 = dropper.fit_transform(df2)
#df.head()

In [445]:
df['translated_content'].nunique()

408

In [446]:
df['priority_score'].nunique()

3

###Applying Label encoding tp convert translated_content to numerical values

In [447]:
from sklearn.preprocessing import LabelEncoder
class FeatureEncoder(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    lb = LabelEncoder()
    X['translated_content'] = lb.fit_transform(X['translated_content'])
    return X
#enc = FeatureEncoder()
#enc.fit_transform(df)
#df.head()

In [449]:
#creating the pipeline
from sklearn.pipeline import Pipeline
pipe = Pipeline([("dropper", NameDropper()), ("encoder" , FeatureEncoder())])

In [450]:
#visualizing the pipeline
from sklearn import set_config
set_config(display="diagram")
pipe

Pipeline(steps=[('dropper', NameDropper()), ('encoder', FeatureEncoder())])

In [451]:
df2 = pipe.fit_transform(df2)
df2 = pd.DataFrame(df2)
#print(type(df2))
#df2.head()
df2

,Unnamed: 0,translated_content,priority_score,problem_domain
0,1,219,2,0
1,2,13,3,1
2,3,38,3,1
3,4,239,3,1
4,5,178,2,0
...,...,...,...,...
695,696,183,2,0
696,697,138,2,0
697,698,379,2,0
698,699,280,2,0


In [452]:
df.shape

(700, 5)

In [453]:
X = df.drop('problem_domain', axis = 1)
y = df['problem_domain']
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)
y_train.head()

328    0
627    1
231    0
226    0
637    0
Name: problem_domain, dtype: int64

In [454]:
X_train = pipe.fit_transform(X_train)
X_train.head()
#X_train.head()
#df.head()

,Unnamed: 0,translated_content,priority_score
328,329,4,2
627,628,184,2
231,232,272,3
226,227,198,2
637,638,113,2


In [455]:
X_train.shape

(560, 3)

In [457]:
X_test = pipe.fit_transform(X_test)
X_test.head()

,Unnamed: 0,translated_content,priority_score
457,458,34,2
345,346,26,3
204,205,26,3
583,584,38,3
180,181,71,1


In [460]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state = 50)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=50)

In [463]:
y_predict = dtc.predict(X_test)
y_predict

array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0])

In [466]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
report = classification_report(y_test, y_predict)
print(report)

              precision    recall  f1-score   support

           0       0.89      0.80      0.84       114
           1       0.39      0.58      0.47        26

    accuracy                           0.76       140
   macro avg       0.64      0.69      0.66       140
weighted avg       0.80      0.76      0.77       140



In [467]:
accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.7571428571428571


In [468]:
cm = confusion_matrix(y_test, y_predict)
cm

array([[91, 23],
       [11, 15]])